In [22]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import numpy as np
import scipy.io as sio

def concatenate_npy_files(file_paths, output_file, output_format):
    if not file_paths:
        messagebox.showerror("错误", "没有选择任何文件。")
        return
    
    arrays = []
    
    for file_path in file_paths:
        data = np.load(file_path, allow_pickle=True)  # 允许加载pickle数据
        arrays.append(data)
    
    concatenated_array = np.concatenate(arrays, axis=0)
    
    if output_format == 'npy':
        np.save(output_file, concatenated_array)
    elif output_format == 'mat':
        sio.savemat(output_file, {'data': concatenated_array})
    
    messagebox.showinfo("成功", f"拼接后的数据已保存到 {output_file}")

def browse_files():
    file_paths = filedialog.askopenfilenames(filetypes=[("NumPy files", "*.npy")])
    if file_paths:
        file_paths_var.set('; '.join(file_paths))

def browse_output_file():
    file_selected = filedialog.asksaveasfilename(defaultextension='.npy',
                                                 filetypes=[("NumPy files", "*.npy"), ("MAT files", "*.mat")])
    
    if file_selected:
        output_file_var.set(file_selected)
        update_output_file_extension()

def update_output_file_extension(*args):
    output_file = output_file_var.get()
    output_format = output_format_var.get()
    
    if output_file:
        if output_format == 'npy' and not output_file.endswith('.npy'):
            output_file += '.npy'
        elif output_format == 'mat' and not output_file.endswith('.mat'):
            output_file += '.mat'
        
        output_file_var.set(output_file)

def on_concatenate():
    file_paths = file_paths_var.get().split('; ')
    output_file = output_file_var.get()
    output_format = output_format_var.get()
    
    if not file_paths or not output_file:
        messagebox.showerror("错误", "请指定要拼接的文件和输出文件路径。")
        return
    
    concatenate_npy_files(file_paths, output_file, output_format)

def on_close():
    root.destroy()

# 创建主窗口
root = tk.Tk()
root.title("拼接 .npy 文件")

# 创建选择文件按钮和显示框
tk.Label(root, text="选择要拼接的文件:").grid(row=0, column=0, padx=10, pady=5)
file_paths_var = tk.StringVar()
tk.Entry(root, textvariable=file_paths_var, width=50, state='readonly').grid(row=0, column=1, padx=10, pady=5)
tk.Button(root, text="浏览", command=browse_files).grid(row=0, column=2, padx=10, pady=5)

# 创建输出文件路径选择按钮和显示框
tk.Label(root, text="输出文件:").grid(row=1, column=0, padx=10, pady=5)
output_file_var = tk.StringVar()
tk.Entry(root, textvariable=output_file_var, width=50).grid(row=1, column=1, padx=10, pady=5)
tk.Button(root, text="浏览", command=browse_output_file).grid(row=1, column=2, padx=10, pady=5)

# 创建保存格式选择下拉菜单
tk.Label(root, text="保存格式:").grid(row=2, column=0, padx=10, pady=5)
output_format_var = tk.StringVar()
output_format_var.set('npy')  # 默认选择 npy 格式
format_menu = ttk.Combobox(root, textvariable=output_format_var, values=['npy', 'mat'], state='readonly')
format_menu.grid(row=2, column=1, padx=10, pady=5)
output_format_var.trace_add('write', update_output_file_extension)

# 创建拼接按钮
tk.Button(root, text="拼接文件", command=on_concatenate).grid(row=3, column=0, columnspan=3, padx=10, pady=10)

# 创建关闭按钮
tk.Button(root, text="关闭", command=on_close).grid(row=4, column=0, columnspan=3, padx=10, pady=10)

# 启动主事件循环
root.mainloop()